In [1]:
import pandas as pd 
import numpy as np 
import pandas_profiling
import datetime
from geopy.geocoders import Nominatim

#Original file 
csv_path = '/Users/chrisrios/Desktop/Data_science/Airbnb_Project/airbnb_data/airbnb_listings.csv'

#Making the Dataframe
airbnb_df = pd.read_csv(csv_path)

#output the dataframe
print(airbnb_df)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/chrisrios/Desktop/Data_science/Airbnb_Project/airbnb_data/airbnb_listings.csv'

In [ ]:
airbnb_df.drop(columns= 
    ['id',
    'listing_url',
    'scrape_id',
    'last_scraped',
    'name',
    'description',
    'neighborhood_overview',
    'picture_url',
    'host_id',
    'host_url',
    'host_name',
    'host_since',
    'host_location',
    'host_about',
    'host_response_time',
    'host_response_rate',
    'host_acceptance_rate',
    'host_is_superhost',
    'host_thumbnail_url',
    'host_picture_url',
    'host_listings_count',
    'host_total_listings_count',
    'host_verifications',
    'host_has_profile_pic',
    'host_identity_verified',
    'neighbourhood',
    'neighbourhood_group_cleansed',
    'property_type',
    'accommodates',
    'bathrooms',
    'beds',
    'amenities',
    'minimum_nights',
    'maximum_nights',
    'minimum_minimum_nights',
    'maximum_minimum_nights',
    'minimum_maximum_nights',
    'maximum_maximum_nights',
    'minimum_nights_avg_ntm',
    'maximum_nights_avg_ntm',
    'calendar_updated',
    'has_availability',
    'availability_30',
    'availability_60',
    'availability_90',
    'availability_365',
    'calendar_last_scraped',
    'number_of_reviews',
    'number_of_reviews_ltm',
    'number_of_reviews_l30d',
    'first_review',
    'last_review',
    'review_scores_rating',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value',
    'license',
    'instant_bookable',
    'calculated_host_listings_count',
    'calculated_host_listings_count_entire_homes',
    'calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms',
    'reviews_per_month',
    'review_scores_accuracy'
    ],inplace=True)

In [ ]:
#Rename columns 
airbnb_df.rename(columns=
{
    'host_neighbourhood':'neighborhood',
    'neighbourhood_cleansed':'zipcode',
    'bathrooms_text':'num_bathrooms',
    'bedrooms':'num_bedrooms',
    'price':'listing_price'
},inplace=True)

#Reorder
airbnb_df = airbnb_df[[
    'longitude',
    'latitude',
    'zipcode',
    'num_bedrooms',
    'num_bathrooms',
    'listing_price',
    'neighborhood'
     ]]

In [ ]:
#Remove text from bathrooms
airbnb_df['num_bathrooms'] = airbnb_df['num_bathrooms'].str.extract('(\d+)', expand=False)

#Remove NaNs from bedrooms (no way to accurately interpolate)
airbnb_df.dropna(subset='num_bedrooms', inplace= True)




In [ ]:
def convertToCategory(*args):
    for col in args:
        airbnb_df[col] = airbnb_df[col].astype('category')

def convertToFloat(*args):
    for col in args:
        airbnb_df[col] = airbnb_df[col].astype('float64')

def replaceCharacters(*args, column, newvalue):
    """Replaces x strings of characters, takes column and the new value to replace the values with """
    for oldvalue in args:
        airbnb_df[column] = airbnb_df[column].str.replace(oldvalue,newvalue)

replaceCharacters('$',',',column='listing_price', newvalue='')

#Change Types
convertToFloat('num_bedrooms','num_bathrooms','listing_price')
convertToCategory('neighborhood')

In [ ]:
#Count missing bathrooms
NanbathroomCount = airbnb_df['num_bathrooms'].isna().sum()
print("There were " + str(NanbathroomCount) + " missing bathrooms")

#Backfill Bedrooms based on bathrooms + zipcode
airbnb_df['num_bathrooms'] = airbnb_df.groupby(['zipcode', 'num_bedrooms']).num_bedrooms.bfill()
Nanbathrooms = airbnb_df['num_bathrooms'].isna().sum()
print("After backfilling there are now " + str(Nanbathrooms) + " missing bathrooms")

In [ ]:
airbnb_df.dtypes

In [ ]:
#Remove outliers
airbnb_df = airbnb_df[ (airbnb_df.listing_price < 9000) & (airbnb_df.listing_price > 15)]

In [ ]:
split_char = ","
geolocator = Nominatim(user_agent="airbnb zipcode analyzer")

for index,row in airbnb_df.iterrows():
        # Grabbing the latitude and longitude from the DF            
        latitude = str(airbnb_df.loc[index , 'latitude'])
        longitude = str(airbnb_df.loc[index , 'longitude'])
        latlong = latitude + ", "+ longitude
            
        #Use geopy to grab address string 
        location = geolocator.reverse(latlong)

        #Grab the street address 
        temp = location[0].split(split_char)
        resdirty =  split_char.join(temp[:2])

        #Clean street address
        resclean = resdirty.replace(",","")

        airbnb_df.loc[index,'street_address'] = resclean

        print(airbnb_df.loc[index,'street_address'])
        print("Currently working on row " + str(index))

In [ ]:
airbnb_df = airbnb_df[[
    'street_address',
    'latitude',
    'longitude',
    'zipcode',
    'num_bedrooms',
    'num_bathrooms',
    'listing_price',
    'neighborhood'
     ]]

In [ ]:
airbnb_df.to_csv('/Users/chrisrios/Desktop/Data_science/Airbnb_Project/airbnb_data/cleanedlistings.csv')